In [1]:
%run Configuration.ipynb

In [2]:
from datetime import date, datetime, timedelta
import os
today = date.today()

In [3]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

try:
  client = MailchimpMarketing.Client()
  client.set_config({
    "api_key": os.getenv('Mailchimp_API_Key'),
    "server": os.getenv('Mailchimp_server')
  })

  response = client.lists.get_list_members_info(os.getenv('Mailchimp_list_id'))
  #print(response)
except ApiClientError as error:
  print("Error: {}".format(error.text))

In [5]:
users_email_address = []
for itr in range(len(response['members'])):
    users_email_address.append(response['members'][itr]['email_address'])

In [7]:
users_email = {"user_email":users_email_address}

In [8]:
import json
with open(r'C:\Users\sreer\Big data project\Stock_results\NSE_{}.json'.format(today), 'r') as fp:
    NSE_data = json.load(fp)

In [9]:
with open(r'C:\Users\sreer\Big data project\Stock_results\Sentiment_result_{}.json'.format(today), 'r') as fp:
    Sentiment_result = json.load(fp)

In [10]:
import smtplib
from email.message import EmailMessage
from email.mime.text import MIMEText
from email.utils import make_msgid
import mimetypes
from email.mime.multipart import MIMEMultipart
EMAIL_ADDRESS = os.getenv('email_address')
EMAIL_PASSWORD = os.getenv('email_password')

In [11]:
msg = EmailMessage()
msg['Subject'] = 'Stock Mail Newsletter dated {}'.format(today)
msg['From'] = EMAIL_ADDRESS 
msg['To'] = users_email_address

msg.set_content("The predicted trend of NIFTY-50 for today is", Sentiment_result["Sentiment_type_of_the_day"])
image_cid = make_msgid(domain='stock_mail.com')
msg.add_alternative(f'''\
<!DOCTYPE html>
<html lang="en" dir="ltr">
  <head>
    <meta charset="utf-8">
    <title>Stock Mail</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@100;500&display=swap" rel="stylesheet">
  </head>
  <body style="font-family: 'Montserrat', sans-serif;">
    <div class="d1" >
      <img class="stock_mail_logo" src="cid:{image_cid[1:-1]}" alt="stock_mail_logo" style="  display: block;margin-left: auto;margin-right: auto;width: 25%;">
    </div>
    <div class="d2" style="text-align: center;font-weight: bold;">
      <h1>Stock Mail</h1>
      
    </div>
    <div class="d3">
    <p>Dear Subscriber,</p>
      <p>Please find the yesterday's NSE NIFTY-50 details below: </p>
      <table id= "NSE" style="border-collapse:collapse;border:1px solid black;text-align:center;table-layout: fixed;width: 500px;">
        <thead style="border:1px solid black;text-align:center;">
          <th style="border:1px solid black;text-align:center;">Index</th>
          <th style="border:1px solid black;text-align:center;">Value</th>
          <th style="border:1px solid black;text-align:center;">Change</th>
          <th style="border:1px solid black;text-align:center;">Percent Change</th>
        </thead>
<tbody>
<tr style="border:1px solid black;text-align:center;">
  <td style="border:1px solid black;text-align:center;">{NSE_data["Index"]}</td>
  <td style="border:1px solid black;text-align:center;">{NSE_data["Value"]}</td>
  <td style="border:1px solid black;text-align:center;">{NSE_data["Change"]}</td>
  <td style="border:1px solid black;text-align:center;">{NSE_data["Percent_Change"]}</td>
</tr>
</tbody>
      </table>
    </div>
<div class="d4">
  <p>Generally, the stock market is so volatile. We could reduce our risk of investing by predicting the  trend.</p><br>
  <p>The trend of stock index is heavily decided by the news that circulate right now.</p><br>
  <p>Stock Mail provides an exclusive information on the trend for the data. </p><br>
</div>
<div class="d5">
  <p>The predicted trend of NIFTY-50 for today is <strong>{Sentiment_result["Sentiment_type_of_the_day"]}</strong></p>
  <p>For more information on the news prediction result click on this link </p><a href="https://storage.googleapis.com/stock_mail/Headlines_db/Headlines_{today}.csv">https://storage.googleapis.com/stock_mail/Headlines_db/Headlines_{today}.csv</a>
    <br>
  <br><br>
  <p>Cheers,</p>
  <p>Stock Mail Team</p>
  
</div>
  </body>
</html>
''', subtype='html')
with open('market_news.jpg', 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD) 
    smtp.send_message(msg)
    #smtp.sendmail(sender_email, receiver_email, msg.as_string())